# 🎬 Notebook Limpo: Geração de Vídeo a partir de Imagem

Este notebook é minimalista e estável:
- End-to-end TXT2IMG (sem dependências de outras células)
- Alternativa: Stable Video Diffusion (gera vídeo diretamente da imagem)

Use este notebook em vez do antigo para evitar conflitos de estado.


In [3]:
# Instalação (execute uma vez por sessão)
%pip -q install diffusers==0.30.0 transformers accelerate safetensors "imageio[ffmpeg]" pillow



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
# Celula Única: vídeo estável só com TXT2IMG
import os, numpy as np
from PIL import Image
import torch, imageio
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler

# Config
model_id = "runwayml/stable-diffusion-v1-5"
width, height = 512, 512
width  = (width  // 8) * 8
height = (height // 8) * 8
steps, guidance, frames, fps = 20, 7.5, 8, 12
negative_prompt = ""

# Imagem do utilizador
img_path = "/Users/alansms/stable_diffusion_video_generator/modelo-1.png"  # altere se necessário
if not os.path.exists(img_path):
    raise FileNotFoundError(f"Imagem não encontrada: {img_path}")
init_img = Image.open(img_path).convert("RGB").resize((width, height), Image.LANCZOS)

# Pipeline
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device=="cuda" else torch.float32,
    safety_checker=None,
    requires_safety_checker=False,
)
try:
    pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
except Exception:
    pass
pipe = pipe.to(device)

# Gerar frames
base_prompt = "cinematic, detailed, continue the theme of a crocodile (jacaré), high quality"
frames_list = [init_img]
cur = init_img
for i in range(1, frames):
    out = pipe(
        prompt=base_prompt,
        negative_prompt=negative_prompt,
        width=width, height=height,
        num_inference_steps=steps,
        guidance_scale=guidance,
    )
    img = out.images[0]
    if not isinstance(img, Image.Image):
        img = Image.fromarray(np.array(img)).convert("RGB")
    frames_list.append(img)
    cur = img
print(f"OK: {len(frames_list)} frames gerados (TXT2IMG).")

# Exportar
os.makedirs("output", exist_ok=True)
out_path = "output/jacare_txt2img_video.mp4"
with imageio.get_writer(out_path, fps=fps, quality=8) as w:
    for im in frames_list:
        w.append_data(np.array(im.convert("RGB")))
print("Vídeo salvo em:", out_path)


100%|██████████| 20/20 [00:54<00:00,  2.72s/it]


OK: 8 frames gerados (TXT2IMG).
Vídeo salvo em: output/jacare_txt2img_video.mp4


In [ ]:
# Versão Inteligente: Análise automática da imagem + retroalimentação
import os, numpy as np
from PIL import Image
import torch, imageio
from diffusers import StableDiffusionImg2ImgPipeline, DPMSolverMultistepScheduler

print("🎬 Iniciando geração inteligente com análise da imagem...")

# Configuração
model_id = "runwayml/stable-diffusion-v1-5"
width, height = 512, 512
width  = (width  // 8) * 8
height = (height // 8) * 8
steps, guidance, frames, fps = 20, 7.5, 12, 12
strength = 0.6  # força da transformação (reduzido para manter consistência)
negative_prompt = "blurry, low quality, distorted, deformed, ugly"

# Carregar imagem inicial
img_path = "/Users/alansms/stable_diffusion_video_generator/jacaré.png"
if not os.path.exists(img_path):
    raise FileNotFoundError(f"Imagem não encontrada: {img_path}")
init_img = Image.open(img_path).convert("RGB").resize((width, height), Image.LANCZOS)
print(f"✅ Imagem inicial carregada: {init_img.size}")

# Pipeline Img2Img
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔧 Inicializando pipeline no dispositivo: {device}")
pipe_img2img = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device=="cuda" else torch.float32,
    safety_checker=None,
    requires_safety_checker=False,
)
try:
    pipe_img2img.scheduler = DPMSolverMultistepScheduler.from_config(pipe_img2img.scheduler.config)
except Exception:
    pass
pipe_img2img = pipe_img2img.to(device)
print("✅ Pipeline Img2Img pronto")

# Prompt genérico que funciona com qualquer imagem
base_prompt = "cinematic, detailed, high quality, professional photography, maintain the original subject and style"
frames_list = [init_img]
current = init_img

print(f"🎞️ Gerando {frames} frames com retroalimentação inteligente...")
print("📝 Usando prompt genérico para preservar o conteúdo original da imagem")

for i in range(1, frames):
    print(f"  Frame {i+1}/{frames}: analisando frame anterior...")
    
    # GERAR PRÓXIMO FRAME usando o anterior como entrada
    out = pipe_img2img(
        prompt=base_prompt,
        negative_prompt=negative_prompt,
        image=current,  # ← RETROALIMENTAÇÃO: usa frame anterior
        strength=strength,  # força reduzida para manter consistência
        num_inference_steps=steps,
        guidance_scale=guidance,
    )
    
    img = out.images[0]
    if not isinstance(img, Image.Image):
        img = Image.fromarray(np.array(img)).convert("RGB")
    
    frames_list.append(img)
    current = img  # ← ATUALIZA para o próximo ciclo
    print(f"    ✅ Frame {i+1} gerado mantendo o tema original")

print(f"🎉 Sucesso: {len(frames_list)} frames gerados com retroalimentação inteligente!")

# Exportar vídeo
os.makedirs("output", exist_ok=True)
out_path = "output/video_inteligente_retroalimentacao.mp4"
print(f"💾 Exportando vídeo: {out_path}")
with imageio.get_writer(out_path, fps=fps, quality=8) as w:
    for im in frames_list:
        w.append_data(np.array(im.convert("RGB")))
print(f"✅ Vídeo salvo: {out_path}")
print(f"📊 Estatísticas: {len(frames_list)} frames, {fps} FPS, {width}x{height}px")
print("🎯 Resultado: Vídeo que mantém o tema original da imagem (mulher, jacaré, etc.)")


In [ ]:
# 🎯 DESAFIO: Retroalimentação Real - Cada frame gera o próximo
import os, numpy as np
from PIL import Image
import torch, imageio
from diffusers import StableDiffusionImg2ImgPipeline, DPMSolverMultistepScheduler

print("🎬 Iniciando geração com retroalimentação real...")

# Configuração
model_id = "runwayml/stable-diffusion-v1-5"
width, height = 512, 512
width  = (width  // 8) * 8
height = (height // 8) * 8
steps, guidance, frames, fps = 20, 7.5, 12, 12
strength = 0.7  # força da transformação (0.5-0.8)
negative_prompt = "blurry, low quality, distorted"

# Carregar imagem inicial
img_path = "/Users/alansms/stable_diffusion_video_generator/jacaré.png"
if not os.path.exists(img_path):
    raise FileNotFoundError(f"Imagem não encontrada: {img_path}")
init_img = Image.open(img_path).convert("RGB").resize((width, height), Image.LANCZOS)
print(f"✅ Imagem inicial carregada: {init_img.size}")

# Pipeline Img2Img (retroalimentação)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔧 Inicializando pipeline no dispositivo: {device}")
pipe_img2img = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device=="cuda" else torch.float32,
    safety_checker=None,
    requires_safety_checker=False,
)
try:
    pipe_img2img.scheduler = DPMSolverMultistepScheduler.from_config(pipe_img2img.scheduler.config)
except Exception:
    pass
pipe_img2img = pipe_img2img.to(device)
print("✅ Pipeline Img2Img pronto")

# Geração com retroalimentação real
base_prompt = "cinematic, detailed, continue the theme and style of the uploaded image, high quality, professional photography"
frames_list = [init_img]
current = init_img

print(f"🎞️ Gerando {frames} frames com retroalimentação...")
for i in range(1, frames):
    print(f"  Frame {i+1}/{frames}: usando frame anterior como entrada...")
    
    # GERAR PRÓXIMO FRAME usando o anterior como entrada
    out = pipe_img2img(
        prompt=base_prompt,
        negative_prompt=negative_prompt,
        image=current,  # ← RETROALIMENTAÇÃO: usa frame anterior
        strength=strength,
        num_inference_steps=steps,
        guidance_scale=guidance,
    )
    
    img = out.images[0]
    if not isinstance(img, Image.Image):
        img = Image.fromarray(np.array(img)).convert("RGB")
    
    frames_list.append(img)
    current = img  # ← ATUALIZA para o próximo ciclo
    print(f"    ✅ Frame {i+1} gerado e adicionado à sequência")

print(f"🎉 Sucesso: {len(frames_list)} frames gerados com retroalimentação real!")

# Exportar vídeo
os.makedirs("output", exist_ok=True)
out_path = "output/jacare_retroalimentacao_real.mp4"
print(f"💾 Exportando vídeo: {out_path}")
with imageio.get_writer(out_path, fps=fps, quality=8) as w:
    for im in frames_list:
        w.append_data(np.array(im.convert("RGB")))
print(f"✅ Vídeo salvo: {out_path}")
print(f"📊 Estatísticas: {len(frames_list)} frames, {fps} FPS, {width}x{height}px")


🎬 Iniciando geração com retroalimentação real...
✅ Imagem inicial carregada: (512, 512)
🔧 Inicializando pipeline no dispositivo: cpu


Couldn't connect to the Hub: (ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: c021e80d-0575-455e-a6e5-b9749e537e36)').
Will try to load from local cache.
Loading pipeline components...: 100%|██████████| 6/6 [00:00<00:00, 17.74it/s]


✅ Pipeline Img2Img pronto
🎞️ Gerando 12 frames com retroalimentação...
  Frame 2/12: usando frame anterior como entrada...


100%|██████████| 14/14 [00:37<00:00,  2.66s/it]


    ✅ Frame 2 gerado e adicionado à sequência
  Frame 3/12: usando frame anterior como entrada...


...

    ✅ Frame 10 gerado e adicionado à sequência
  Frame 11/12: usando frame anterior como entrada...


100%|██████████| 14/14 [00:36<00:00,  2.64s/it]


    ✅ Frame 11 gerado e adicionado à sequência
  Frame 12/12: usando frame anterior como entrada...


100%|██████████| 14/14 [00:37<00:00,  2.67s/it]


    ✅ Frame 12 gerado e adicionado à sequência
🎉 Sucesso: 12 frames gerados com retroalimentação real!
💾 Exportando vídeo: output/jacare_retroalimentacao_real.mp4
✅ Vídeo salvo: output/jacare_retroalimentacao_real.mp4
📊 Estatísticas: 12 frames, 12 FPS, 512x512px
